SDO/HMI Vector Magnetic Field

The vector magnetic field is derived from full-disk filtergrams collected in a 135-second sequence. (90 seconds after 13 April 2016). The observations are obtained in polarized light at several wavelengths across a spectral line. The filtergrams are calibrated and registered to account for solar rotation before being combined to provide Stokes polarization parameters. The parameters are used to determine the magnetic and other plasma parameters using VFISV, a Milne-Eddington inversion code. The final step is to disambiguate the data, that is to infer the azimuthal direction of the transverse field.

- https://hpde.io/NASA/NumericalData/SDO/HMI/Vector_Magnetic_Field/PT720S.html

In [ ]:
import os
import s3fs
import zarr

from typing import Union

import numpy as np
import dask.array as da
import astropy.units as u
from sunpy.net import Fido, attrs as a
from sunpy.map import Map


def s3_connection(path_to_zarr: os.path) -> s3fs.S3Map:
    """
    Instantiate connection to aws for a given path `path_to_zarr`
    """
    return s3fs.S3Map(
        root=path_to_zarr,
        s3=s3fs.S3FileSystem(anon=True),
        # anonymous access requires no credentials
        check=False,
    )


def load_single_aws_zarr(
    path_to_zarr: os.path,
    cache_max_single_size: int = None,
) -> Union[zarr.Array, zarr.Group]:
    """
    load zarr from s3 using LRU cache
    """
    return zarr.open(
        zarr.LRUStoreCache(
            store=s3_connection(path_to_zarr),
            max_size=cache_max_single_size,
        ),
        mode="r",
    )

In [ ]:
root = load_single_aws_zarr(
    path_to_zarr="s3://gov-nasa-hdrl-data1/contrib/fdl-sdoml/fdl-sdoml-v2/sdomlv2_hmi.zarr",
)
data_bx = root["2012"]["Bx"]
sorted_indices_bx = np.argsort(data_bx.attrs["T_OBS"])

data_by = root["2012"]["By"]
sorted_indices_by = np.argsort(data_by.attrs["T_OBS"])

data_bz = root["2012"]["Bz"]
sorted_indices_bz = np.argsort(data_bz.attrs["T_OBS"])

In [ ]:
def get_map(data, sorted_indices, idx):
    img_index = sorted_indices[idx]
    selected_headr = {keys: values[img_index] for keys, values in data.attrs.items()}
    selected_image = da.from_array(data_bx)[img_index, :, :]
    smap_ml = Map((np.array(selected_image), selected_headr))
    return smap_ml

In [ ]:
ix, iy, iz = 0, 0, 0
smap_ml_bx = get_map(data_bx, sorted_indices_bx, ix)
smap_ml_by = get_map(data_by, sorted_indices_by, iy)
smap_ml_bz = get_map(data_bz, sorted_indices_bz, iz)
print(smap_ml_bx.meta["T_OBS"])
print(smap_ml_by.meta["T_OBS"])
print(smap_ml_bz.meta["T_OBS"])

2011.12.31_23:59:52_TAI
2011.12.31_23:59:52_TAI
2011.12.31_23:59:52_TAI


In [ ]:
tr = a.Time("2012-01-01T00:00:00", "2012-01-01T00:01:00")
hmi_query = Fido.search(tr & a.Instrument.hmi & a.Physobs.vector_magnetic_field)
hmi_query

Start Time,Source,Instrument,Wavelength,Provider,Physobs,Wavetype,Extent Type,Size,Datatype
,,,Angstrom,,,,,Mibyte,
Time,str3,str3,float64[2],str3,str21,str4,str8,float64,str5
2012-01-01 00:00:00.000,SDO,HMI,6173.0 .. 6173.0,NSO,VECTOR_MAGNETIC_FIELD,LINE,SYNOPTIC,2.1377,IMAGE
2012-01-01 00:00:00.000,SDO,HMI,6173.0 .. 6173.0,NSO,VECTOR_MAGNETIC_FIELD,LINE,SYNOPTIC,2.14062,IMAGE


In [ ]:
filelist_fd = Fido.fetch(hmi_query, path="data/{instrument}")

Files Downloaded:   0%|          | 0/2 [00:00<?, ?file/s]

x9x93120101t000000_map_err_qsd_ract_dim_180_source_SDO_HMI_type_b_720s_helio.fits.gz:   0%|          | 0.00/2.…

x9x93120101t000000_map_err_qsd_rand_dim_180_source_SDO_HMI_type_b_720s_helio.fits.gz:   0%|          | 0.00/2.…